In [1]:
import pandas as pd

# RAIS

## Estabelecimentos


TODO

- Pipeline

    - Leitura
    - Tratamento
    - Merging
    - Load

ano, uf, meso, micro, muni

In [9]:
def read_data(caminho, ano):
    try: # txt file
        colunas = ['CNAE 2.0 Classe', 'Município']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.txt', sep=';', encoding='latin1', usecols=colunas)
        df_estb['ano'] = ano
        extensao_arquivo = 'txt'
        
    except: # csv file
        colunas = ['ano', 'id_municipio', 'cnae_2']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.csv', usecols=colunas)
        extensao_arquivo = 'csv'

    return df_estb, extensao_arquivo

In [18]:
def transform_data(df, extensao):
    '''Normaliza o tipo de cada coluna e obtem a cnae_secao de cnae_2'''
    
    if extensao == 'txt':
        df.rename(columns={'Município': 'id_municipio', 'CNAE 2.0 Classe': 'cnae_2'}, inplace=True)	
        df = df[['ano', 'id_municipio', 'cnae_2']]
    elif extensao == 'csv':
        df.dropna(inplace=True)
        df['id_municipio'] = df['id_municipio'].astype('int')
        df['id_municipio'] = df['id_municipio'].apply(lambda x: int(x/10)) # remove 7'th dígito
    
    # para todos
    df['ano'] = df['ano'].astype('int16')
    
    df['id_municipio'] = df['id_municipio'].astype('int')
        
    df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
    df['cnae_secao'] = df['cnae_2'].apply(lambda x: x[:2])
    df['cnae_secao'] = df['cnae_secao'].astype('int')
    df.drop(columns=['cnae_2'], inplace=True)
    return df

In [11]:
def merge_municipios(df, caminho):
    '''Faz o merge do id_municipio com os dados do dicionário'''
    
    colunas_muni = ['id_municipio_6', 'nome', 'nome_microrregiao', 'nome_mesorregiao', 'sigla_uf', 'nome_uf', 'nome_regiao']
    dicionario_muni = pd.read_csv(caminho, sep=',', usecols=colunas_muni)
    dicionario_muni.rename(columns={'id_municipio_6': 'id_municipio', 'nome': 'municipio'}, inplace=True)
    
    df = pd.merge(df, dicionario_muni, on='id_municipio', how='left')
    df.drop(columns=['id_municipio'], inplace=True)
    return df

In [12]:
def merge_cnae(df, caminho):
    
    colunas_tmp = ['divisao','descricao_secao']
    df_cnae = pd.read_csv(caminho, usecols=colunas_tmp)
    df_cnae.rename(columns={'divisao': 'cnae_secao'}, inplace=True)
    df_cnae.drop_duplicates(inplace=True)
    
    df = pd.merge(df, df_cnae, on='cnae_secao', how='left')
    df.drop(columns=['cnae_secao'], inplace=True)
    return df

In [48]:
def calc_ql(df, loc='meso'):
    '''Calcula o quociente de localização para níveis de meso, microrregião ou município'''
    
    if loc == 'meso':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'descricao_secao', 'quociente_localizacao']
    elif loc == 'micro':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'descricao_secao', 'quociente_localizacao']
    elif loc == 'muni':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'municipio', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'municipio']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'municipio', 'descricao_secao', 'quociente_localizacao']

    ql['quociente_localizacao'] = ql['quociente_localizacao'].map(lambda x: float(f"{x:.2f}"))

    return ql


In [50]:
def etl_pipeline(caminhos, ano_ini, ano_fim, loc):
    dfs_ql = []
    for ano in range(ano_ini, ano_fim+1):
        print('---', ano, '---')
        df, extensao = read_data(caminhos['rais'], str(ano))
        df = transform_data(df, extensao)
        df = merge_municipios(df, caminhos['municipios'])
        df = merge_cnae(df, caminhos['cnae'])
        
        dfs_ql.append(calc_ql(df, loc))
    df = pd.concat(dfs_ql)
    return df

---

In [59]:
caminho = "D:/dados/rais/estabelecimentos/"
caminho_dict_muni = "D:/dados/rais/dicionarios/dicionario_municipios.csv"
caminho_dict_cnae = "D:/dados/rais/dicionarios/dicionario_cnae_2.csv"
ano_ini = 2023
ano_fim = 2023
caminhos = {'rais': caminho, 
            'municipios': caminho_dict_muni, 
            'cnae': caminho_dict_cnae}

df_ql_muni = etl_pipeline(caminhos, ano_ini, ano_fim, 'muni')
df_ql_muni

--- 2023 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_20772\844555952.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['47890' '42995' '94308' ... '47822' '43118' '11135']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))


,ano,sigla_uf,nome_mesorregiao,nome_microrregiao,municipio,descricao_secao,quociente_localizacao
0,2023,AC,Vale do Acre,Brasiléia,Assis Brasil,"Administração Pública, Defesa E Seguridade Social",0.64
1,2023,AC,Vale do Acre,Brasiléia,Brasiléia,"Administração Pública, Defesa E Seguridade Social",0.18
2,2023,AC,Vale do Acre,Brasiléia,Epitaciolândia,"Administração Pública, Defesa E Seguridade Social",0.86
3,2023,AC,Vale do Acre,Brasiléia,Xapuri,"Administração Pública, Defesa E Seguridade Social",1.92
4,2023,AC,Vale do Acre,Rio Branco,Acrelândia,"Administração Pública, Defesa E Seguridade Social",2.56
...,...,...,...,...,...,...,...
93186,2023,TO,Oriental do Tocantins,Porto Nacional,Palmas,"Água, Esgoto, Atividades De Gestão De Resíduos...",0.96
93187,2023,TO,Oriental do Tocantins,Porto Nacional,Pedro Afonso,"Água, Esgoto, Atividades De Gestão De Resíduos...",0.69
93188,2023,TO,Oriental do Tocantins,Porto Nacional,Porto Nacional,"Água, Esgoto, Atividades De Gestão De Resíduos...",1.36
93189,2023,TO,Oriental do Tocantins,Porto Nacional,Santa Maria do Tocantins,"Água, Esgoto, Atividades De Gestão De Resíduos...",4.71


In [61]:
df_ql_muni_mg = df_ql_muni[df_ql_muni['sigla_uf'] == 'MG']
df_ql_muni_mg.sort_values(by='quociente_localizacao', ascending=False)

,ano,sigla_uf,nome_mesorregiao,nome_microrregiao,municipio,descricao_secao,quociente_localizacao
35299,2023,MG,Vale do Rio Doce,Aimorés,Taparuba,Serviços Domésticos,104.48
35286,2023,MG,Sul/Sudoeste de Minas,São Lourenço,Alagoa,Serviços Domésticos,81.26
32638,2023,MG,Vale do Mucuri,Teófilo Otoni,Franciscópolis,Indústrias Extrativas,59.52
31091,2023,MG,Noroeste de Minas,Unaí,Arinos,Eletricidade E Gás,51.55
35266,2023,MG,Norte de Minas,Montes Claros,Lontra,Serviços Domésticos,49.52
...,...,...,...,...,...,...,...
23104,2023,MG,Metropolitana de Belo Horizonte,Belo Horizonte,Vespasiano,"Agricultura, Pecuária, Produção Florestal, Pes...",0.05
27562,2023,MG,Zona da Mata,Muriaé,Divino,Atividades Imobiliárias,0.04
23096,2023,MG,Metropolitana de Belo Horizonte,Belo Horizonte,Raposos,"Agricultura, Pecuária, Produção Florestal, Pes...",0.03
23086,2023,MG,Metropolitana de Belo Horizonte,Belo Horizonte,Contagem,"Agricultura, Pecuária, Produção Florestal, Pes...",0.02


---